In [ ]:
!pip install selenium webdriver-manager

In [ ]:
!pip install pandas

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import urllib.parse
import pandas as pd
import os
import time
import json
import datetime
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [2]:
    #  "Python": "Django, Flask, FastAPI, PyTorch",
    # "Java": "Spring Boot, Hibernate, Struts",
    # "JavaScript": "React, Vue, Node.js, Next.js",
    # "C#": ".NET, Unity, Xamarin",
    # "PHP": "Laravel, Symfony, CodeIgniter",
    # "Visual Basic": "VBA, .NET Framework",
    #     //*
# "C++": "Qt, Boost, Unreal Engine,Embedded C, Win32 API",
#         "SQL": "PostgreSQL, MySQL, T-SQL",
#         "Assembly language": "NASM, MASM",
#         "SQL": "SQL Server, PostgreSQL",
#         "Swift": "SwiftUI, UIKit, Vapor",
#         "Rust": "Rocket, Actix, Warp",
# "Fortran": "LAPACK, Scipy (Binding)",
#       "Delphi/Object Pascal": "VCL, FireMonkey",
#       "MATLAB": "Simulink, Deep Learning Toolbox",
#       "Delphi/Object Pascal": "VCL, FireMonkey",
#       "Kotlin": "Ktor, Spring Boot, Android Jetpack",
#       "Ruby": "Ruby on Rails, Sinatra",
#       "TypeScript": "Angular, NestJS, Svelte",
#       "COBOL": "Mainframe Batch, CICS",
#       "Scratch": "Block-based Programming",
#       "Dart": "Flutter",  
#       "Julia": "Flux, Genie",
#       "Lisp": "Common Lisp, Emacs Lisp",
#       "Scala": "Akka, Play Framework, Spark",
#       "Prolog": "SWI-Prolog",
#       "Lua": "Löve, Roblox SDK",
#       "Perl": "Catalyst, Dancer",
#       "Objective-C": "Cocoa, Foundation",
#       "Haskell": "Yesod, Servant",
#       "VBScript": "ASP Classic, QTP",
#       "F#": "Giraffe, Fable",
#       "Groovy": "Grails, Gradle",
#       "PL/SQL": "Oracle Database",
#       "Transact-SQL": "MS SQL Server",
#       "Classic Visual Basic": "VB6",
#       "Erlang": "Phoenix (via Elixir), OTP",
#       "Elixir": "Phoenix, Nerves",
# "Clojure": "Luminus, Ring",
#       "Smalltalk": "Pharo, Seaside",
#       "Golang": "Gin, Echo, Fiber",
#       "R-lang": "Tidyverse, Shiny, ggplot2",
#       "Ada": "GNAT, Spark",
#       "Logo": "Turtle Graphics",
#       "ABAP": "SAP NetWeaver",
#       "Scheme": "Racket, Guile",
    #     
languages_data = {
      "Dlang": "Phobos, Vibe.d",
      "Apex": "Salesforce Framework",
      "Tcl": "Tk"
}
current_page = 0
data_all = []
for i, (lang,frame) in enumerate(languages_data.items()):
  framework_list = [f.strip() for f in frame.split(',')]
file_name = 'Raw_data_AddYear_HackerNews.json'

In [3]:
import json
current_page = 0
data_all = []
output_dir = '../raw_data'
output_path = os.path.join(output_dir, file_name)
os.path.exists(output_path)
with open(output_path, 'r', encoding='utf-8') as f:
    data_all = json.load(f)
print(f'Raw data: {len(data_all)} record')

Raw data: 30918 record


In [4]:

for i, (lang,frame) in enumerate(languages_data.items()):
    while True:
        try:
            if (lang == 'C++') or (lang == 'C#') or (lang == 'F#'):
                safe_lang = urllib.parse.quote(lang)
                url = f'https://hn.algolia.com/?dateEnd=1770422400&dateRange=custom&dateStart=1609459200&page={current_page}&prefix=true&query=%{safe_lang}%&sort=byPopularity&type=story'
            else:
                url = f'https://hn.algolia.com/?dateEnd=1770422400&dateRange=custom&dateStart=1609459200&page={current_page}&prefix=true&query=%{lang}%&sort=byPopularity&type=story'
            # print("opening browser...")
            driver.get(url)

            time.sleep(5)

            container = driver.find_elements(By.CLASS_NAME, "Story_container")
            if not container:
                break
            elif current_page == 30:
                break
            # elif lang == 'skipDlang':
            #     break
            # print(f"Search Completed Page {current_page}. Waiting for results...")
            time.sleep(5)
            for item in container:
                try:
                    title = item.find_element(By.CLASS_NAME, "Story_title").find_element(By.TAG_NAME, "a").text
                    meta_section = item.find_element(By.CLASS_NAME, "Story_meta")
                    time_text = meta_section.text
                    scraping_date = datetime.datetime.now()
                    data_all.append({"Title" : title, "language" : lang, "Years" : time_text, "Scraping_Date" : scraping_date})
                except Exception as e:
                    continue
            if data_all:
                output_dir = '../raw_data'
                output_path = os.path.join(output_dir, file_name)
                os.makedirs(output_dir, exist_ok=True) #create dir if not exits
                if not os.path.exists(output_path):
                    with open(output_path, 'w', encoding='utf-8') as f:
                        json.dump(data_all, f, indent=4, ensure_ascii=False, default=str)
                else:
                    with open(output_path, 'w', encoding='utf-8') as f:
                        json.dump(data_all, f, indent=4, ensure_ascii=False, default=str)
            current_page += 1
        except Exception as e:
            print(f"Error: {e}")
    current_page = 0
    framework_list = [f.strip() for f in frame.split(',')]
    for fw in framework_list:
        if (lang == 'C++') or (lang == 'C#') or (lang == 'F#'):
            safe_lang = urllib.parse.quote(lang)
            new_key = safe_lang + " " + fw
        else:
            new_key = lang + " " +  fw
        while True:
            try:
                url = f'https://hn.algolia.com/?dateEnd=1770422400&dateRange=custom&dateStart=1609459200&page={current_page}&prefix=true&query=%{new_key}%&sort=byPopularity&type=story'
                # print("opening browser...")
                driver.get(url)

                time.sleep(5)

                container = driver.find_elements(By.CLASS_NAME, "Story_container")
                if not container:
                    break
                elif current_page == 30:
                    break
                # print(f"Search Completed Page {current_page}. Waiting for results...")
                time.sleep(5)
                for item in container:
                    try:
                        title = item.find_element(By.CLASS_NAME, "Story_title").find_element(By.TAG_NAME, "a").text
                        meta_section = item.find_element(By.CLASS_NAME, "Story_meta")
                        time_text = meta_section.text
                        scraping_date = datetime.datetime.now()
                        data_all.append({"Title" : title, "language" : lang, "Years" : time_text, "Scraping_Date" : scraping_date})
                    except Exception as e:
                        continue
                if data_all:
                    output_dir = '../raw_data'
                    output_path = os.path.join(output_dir, file_name)
                    os.makedirs(output_dir, exist_ok=True) #create dir if not exits
                    if not os.path.exists(output_path):
                        with open(output_path, 'w', encoding='utf-8') as f:
                            json.dump(data_all, f, indent=4, ensure_ascii=False, default=str)
                    else:
                        with open(output_path, 'w', encoding='utf-8') as f:
                            json.dump(data_all, f, indent=4, ensure_ascii=False, default=str)
                current_page += 1
            except Exception as e:
                print(f"Error: {e}")
        current_page = 0
# df_raw_data = pd.DataFrame(data_all).drop_duplicates() #delete same row
# df_raw_data.to_json("Raw_data_HackerNews.json", orient='records', indent=4, force_ascii=False)
# print("Success to JSON file")


In [ ]:
for i, (lang,frame) in enumerate(languages_data.items()):
    if (lang == 'C++') or (lang == 'C#') or (lang == 'F#'):
        safe_lang = urllib.parse.quote(lang)
        print(safe_lang)

In [ ]:
languages_data = {
    "Python": "Django"
}
current_page = 0
data_all = []
for i, (lang,frame) in enumerate(languages_data.items()):
  framework_list = [f.strip() for f in frame.split(',')]
file_name = 'Test_ing.json'

In [ ]:
for i, (lang,frame) in enumerate(languages_data.items()):
    while True:
        try:
            if (lang == 'C++') or (lang == 'C#') or (lang == 'F#'):
                safe_lang = urllib.parse.quote(lang)
                url = f'https://hn.algolia.com/?dateEnd=1770422400&dateRange=custom&dateStart=1609459200&page={current_page}&prefix=true&query=%{safe_lang}%&sort=byPopularity&type=story'
            else:
                url = f'https://hn.algolia.com/?dateEnd=1770422400&dateRange=custom&dateStart=1609459200&page={current_page}&prefix=true&query=%{lang}%&sort=byPopularity&type=story'
            # print("opening browser...")
            driver.get(url)

            time.sleep(5)

            container = driver.find_elements(By.CLASS_NAME, "Story_container")
            if not container:
                break
            if current_page == 2:
                break
            time.sleep(5)
            for item in container:
                try:
                    # print("Add data in data_all")
                    title = item.find_element(By.CLASS_NAME, "Story_title").find_element(By.TAG_NAME, "a").text
                    meta_section = item.find_element(By.CLASS_NAME, "Story_meta")
                    time_text = meta_section.text
                    scraping_date = datetime.datetime.now()
                    data_all.append({"Title" : title, "language" : lang, "Years" : time_text, "Scraping_Date" : scraping_date})
                except Exception as e:
                    continue
            if data_all:
                    output_dir = '../raw_data'
                    output_path = os.path.join(output_dir, file_name)
                    os.makedirs(output_dir, exist_ok=True) #create dir if not exits
                    if not os.path.exists(output_path):
                        with open(output_path, 'w', encoding='utf-8') as f:
                            json.dump(data_all, f, indent=4, ensure_ascii=False)
                    else:
                        with open(output_path, 'w', encoding='utf-8') as f:
                            json.dump(data_all, f, indent=4, ensure_ascii=False)
            current_page += 1
        except Exception as e:
            print(f"Error: {e}")
    current_page = 0
            

In [ ]:
print(data_all)

In [ ]:
import json
current_page = 0
data_all = []
file_name = 'Raw_data_HackerNews.json'
with open(file_name, 'r', encoding='utf-8') as f:
    data_all = json.load(f)
print(f'Raw data: {len(data_all)} record')

In [ ]:
if data_all:
        output_dir = '../raw_data'
        os.makedirs(output_dir, exist_ok=True) 
        output_path = os.path.join(output_dir, file_name)
                
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(data_all, f, indent=4, ensure_ascii=False)

In [ ]:
for i, (lang,frame) in enumerate(languages_data.items()):
        if (lang == 'C++') or (lang == 'C#') or (lang == 'F#'):
            safe_lang = urllib.parse.quote(lang)
            print(safe_lang)

In [ ]:
print(datetime.datetime.now())